# Claude를 활용한 분류: 보험 고객 지원 티켓 분류기

이 가이드에서는 보험 고객 지원 티켓을 10개 카테고리로 분류하는 고정확도 분류 시스템을 구축합니다. 프롬프트 엔지니어링, 검색 증강 생성(RAG), 그리고 Chain-of-Thought 추론을 결합하여 분류 정확도를 70%에서 95% 이상으로 점진적으로 향상시키는 방법을 배웁니다.

이 가이드를 마치면 복잡한 비즈니스 규칙을 처리하고, 제한된 학습 데이터로 작동하며, 설명 가능한 결과를 제공하는 분류 시스템을 설계하는 방법을 이해하게 됩니다.

## 사전 요구사항
- Python 3.11+ 기본 지식
- Anthropic API 키 ([여기서 발급](https://console.anthropic.com))
- VoyageAI API 키 (선택사항 - 임베딩은 미리 계산되어 있음)
- 분류 문제에 대한 이해

## 환경 설정

먼저 필요한 패키지들을 설치합니다.

- anthropic 
- voyageai
- pandas
- matplotlib
- sklearn
- numpy

환경변수에서 API 키를 로드하고 모델명을 설정합니다.

In [1]:
%%capture
!pip install -U anthropic voyageai pandas numpy matplotlib scikit-learn

In [2]:
# API 키 설정 및 클라이언트 재생성
import os
os.environ["ANTHROPIC_API_KEY"] = "REDACTED_ANTHROPIC_KEY"
os.environ["VOYAGE_API_KEY"] = "REDACTED_VOYAGE_KEY"

import anthropic
client = anthropic.Anthropic(api_key=os.environ["ANTHROPIC_API_KEY"])
MODEL = "claude-haiku-4-5"

# 연결 테스트
response = client.messages.create(
    model=MODEL,
    max_tokens=10,
    messages=[{"role": "user", "content": "Hi"}]
)
print("✅ API 연결 성공:", response.content[0].text)
print(f"Model: {MODEL}")

✅ API 연결 성공: # Hello! 👋

How can I
Model: claude-haiku-4-5


## Claude를 활용한 분류

대규모 언어 모델(LLM)은 특히 기존 머신러닝 시스템이 어려움을 겪던 분야에서 분류 영역을 혁신했습니다. LLM은 복잡한 비즈니스 규칙이 있는 분류 문제와 저품질 또는 제한된 학습 데이터 시나리오에서 놀라운 성공을 보여주었습니다. 또한 LLM은 자신의 행동에 대한 자연어 설명과 근거를 제공할 수 있어 분류 과정의 해석 가능성과 투명성을 향상시킵니다. LLM의 힘을 활용하면 기존 머신러닝 접근 방식의 한계를 넘어 데이터가 부족하거나 비즈니스 요구사항이 복잡한 시나리오에서 뛰어난 분류 시스템을 구축할 수 있습니다.

이 가이드에서는 LLM을 활용하여 고급 분류 작업을 수행하는 방법을 살펴봅니다. 다음 핵심 구성요소와 단계를 다룹니다:

1. **데이터 준비**: 학습 데이터와 테스트 데이터를 준비합니다. 학습 데이터는 분류 모델을 구축하는 데 사용되고, 테스트 데이터는 성능을 평가하는 데 활용됩니다. 적절한 데이터 준비는 분류 시스템의 효과를 보장하는 데 매우 중요합니다.

2. **프롬프트 엔지니어링**: 프롬프트 엔지니어링은 분류를 위해 LLM의 힘을 활용하는 데 핵심적인 역할을 합니다. 분류에 사용되는 프롬프트의 구조와 형식을 정의하는 프롬프트 템플릿을 설계합니다. 프롬프트 템플릿에는 사용자 쿼리, 클래스 정의, 벡터 데이터베이스에서 가져온 관련 예시가 포함됩니다. 프롬프트를 신중하게 작성하면 LLM이 정확하고 맥락에 맞는 분류를 생성하도록 유도할 수 있습니다.

3. **검색 증강 생성(RAG) 구현**: 분류 과정을 향상시키기 위해 벡터 데이터베이스를 사용하여 학습 데이터의 임베딩을 저장하고 효율적으로 검색합니다. 벡터 데이터베이스는 유사성 검색을 가능하게 하여 주어진 쿼리에 가장 관련 있는 예시를 찾을 수 있게 합니다. 검색된 예시로 LLM을 보강하면 추가 맥락을 제공하고 생성된 분류의 정확도를 향상시킬 수 있습니다.

4. **테스트 및 평가**: 분류 시스템이 구축되면 변환된 테스트 데이터를 사용하여 성능을 철저히 테스트합니다. 테스트 쿼리를 반복하고, 분류 함수를 사용하여 각 쿼리를 분류하고, 예측된 카테고리와 예상 카테고리를 비교합니다. 분류 결과를 분석하면 시스템의 효과를 평가하고 개선이 필요한 영역을 식별할 수 있습니다.

## 문제 정의: 보험 고객 지원 티켓 분류기

*참고: 이 예제에서 사용된 문제 정의, 데이터, 레이블은 Claude 3 Opus가 합성 생성한 것입니다*

보험 산업에서 고객 지원은 고객 만족과 유지를 보장하는 데 핵심적인 역할을 합니다. 보험 회사는 매일 청구, 정책 관리, 청구 지원 등 다양한 주제를 다루는 대량의 지원 티켓을 받습니다. 이러한 티켓을 수동으로 분류하는 것은 시간이 많이 걸리고 비효율적이어서 응답 시간이 길어지고 고객 경험에 잠재적으로 영향을 미칠 수 있습니다.

#### 카테고리 정의

1. Billing Inquiries (청구 문의)
- 청구서, 요금, 수수료, 보험료에 대한 질문
- 청구 명세서에 대한 설명 요청
- 결제 방법 및 납부 기한에 대한 문의

2. Policy Administration (정책 관리)
- 정책 변경, 업데이트 또는 취소 요청
- 정책 갱신 및 복원에 대한 질문
- 보장 옵션 추가 또는 제거에 대한 문의

3. Claims Assistance (청구 지원)
- 청구 절차 및 제출 방법에 대한 질문
- 청구 문서 제출에 대한 도움 요청
- 청구 상태 및 지급 일정에 대한 문의

4. Coverage Explanations (보장 범위 설명)
- 특정 정책 유형에서 보장되는 내용에 대한 질문
- 보장 한도 및 제외 사항에 대한 설명 요청
- 공제액 및 본인 부담 비용에 대한 문의

5. Quotes and Proposals (견적 및 제안)
- 새 정책 견적 및 가격 비교 요청
- 이용 가능한 할인 및 번들 옵션에 대한 질문
- 다른 보험사에서 전환하는 것에 대한 문의

6. Account Management (계정 관리)
- 로그인 자격 증명 또는 비밀번호 재설정 요청
- 온라인 계정 기능에 대한 질문
- 연락처 또는 개인 정보 업데이트에 대한 문의

7. Billing Disputes (청구 분쟁)
- 예상치 못한 또는 잘못된 청구에 대한 불만
- 환불 또는 보험료 조정 요청
- 연체료 또는 추심 통지에 대한 문의

8. Claims Disputes (청구 이의)
- 거부되거나 과소 지급된 청구에 대한 불만
- 청구 결정 재고 요청
- 청구 결과에 대한 이의 제기 문의

9. Policy Comparisons (정책 비교)
- 정책 옵션 간의 차이점에 대한 질문
- 보장 수준 선택에 대한 도움 요청
- 경쟁사 상품과의 비교에 대한 문의

10. General Inquiries (일반 문의)
- 회사 연락처 정보 또는 영업시간에 대한 질문
- 제품 또는 서비스에 대한 일반적인 정보 요청
- 다른 카테고리에 명확히 맞지 않는 문의

## 데이터 로딩 및 준비

문제와 카테고리를 정의했으니, 레이블이 지정된 학습 데이터와 테스트 데이터를 로드합니다. TSV 파일을 읽고, 구조화된 형식으로 변환하고, 평가를 위한 테스트 세트를 준비합니다.

학습 데이터에는 나중에 검색 증강 생성(RAG)에 사용할 68개의 레이블된 예시가 포함되어 있습니다. 테스트 세트에도 분류 접근 방식을 평가하는 데 사용할 68개의 예시가 포함되어 있습니다.

다음 데이터셋을 사용합니다:
- `./data/test.tsv`
- `./data/train.tsv`

## Loading and Preparing the Data

Now that we've defined our problem and categories, let's load the labeled training and test data. We'll read the TSV files, convert them into a structured format, and prepare our test set for evaluation.

The training data contains 68 labeled examples that we'll use for retrieval-augmented generation (RAG) later. The test set also contains 68 examples that we'll use to evaluate our classification approaches.

We will use the following datasets:
- `./data/test.tsv`
- `./data/train.tsv`

## 평가 프레임워크

분류기를 구축하기 전에 성능을 측정하기 위한 평가 프레임워크를 설정합니다. 두 가지 핵심 함수를 만듭니다:

1. **`evaluate(X, y, classifier, batch_size)`**: 동시 실행을 사용하여 모든 테스트 예시에 대해 분류기를 실행하고, 정확도 메트릭을 계산하고, 혼동 행렬을 생성합니다. `batch_size` 매개변수는 동시에 수행할 API 요청 수를 제어합니다.

2. **`plot_confusion_matrix(cm, labels)`**: 혼동 행렬을 시각화하여 어떤 카테고리들이 서로 혼동되고 있는지 보여주며, 분류기가 어려움을 겪는 부분을 식별하는 데 도움을 줍니다.

이 프레임워크를 통해 다양한 분류 접근 방식을 경험적으로 비교하고 전체 정확도뿐만 아니라 어떤 특정 카테고리가 어려운지 이해할 수 있습니다.

**Rate Limits**: `MAXIMUM_CONCURRENT_REQUESTS`는 기본적으로 1로 설정되어 있습니다. 더 높은 rate limit 티어가 있다면 이 값을 증가시켜 평가 속도를 높일 수 있습니다. 티어별 제한에 대해서는 [rate limits 문서](https://docs.claude.com/en/api/rate-limits)를 참조하세요.

## Evaluation Framework

Before we build our classifiers, let's set up an evaluation framework to measure their performance. We'll create two key functions:

1. **`evaluate(X, y, classifier, batch_size)`**: Runs your classifier on all test examples using concurrent execution for speed, then calculates accuracy metrics and generates a confusion matrix. The `batch_size` parameter controls how many concurrent API requests to make.

2. **`plot_confusion_matrix(cm, labels)`**: Visualizes the confusion matrix to show which categories are being confused with each other, helping identify where the classifier struggles.

This framework allows us to empirically compare different classification approaches and understand not just overall accuracy, but which specific categories are challenging.

**Rate Limits**: The `MAXIMUM_CONCURRENT_REQUESTS` is set to 1 by default. If you have a higher rate limit tier, you can increase this value to speed up evaluation. See [rate limits documentation](https://docs.claude.com/en/api/rate-limits) for your tier's limits.

## 기준선: 랜덤 분류기

성능 기준선을 설정하고 평가 프레임워크가 올바르게 작동하는지 확인하기 위해, 각 티켓에 대해 무작위로 카테고리를 선택하는 랜덤 분류기로 시작합니다.

이것은 성능의 하한선을 제공합니다: 어떤 실제 분류 접근 방식이든 무작위 추측(10개 카테고리에서 약 10%의 정확도를 달성해야 함)보다 훨씬 나은 성능을 보여야 합니다. 이 기준선은 우리의 컨텍스트 엔지니어링이 실제로 얼마나 많은 가치를 추가하는지 정량화하는 데 도움이 됩니다.

## Baseline: Random Classifier

To establish a performance baseline and verify our evaluation framework works correctly, let's start with a random classifier that picks a category at random for each ticket.

This gives us a lower bound on performance: any real classification approach should significantly outperform random guessing (which should achieve roughly 10% accuracy with 10 categories). This baseline helps us quantify how much value our context engineering actually adds.

In [6]:
import random


def random_classifier(text):
    return random.choice(labels)

무작위 추측에서 예상한 대로, 혼동 행렬은 모든 카테고리에 걸쳐 예측이 무의미한 패턴 없이 흩어져 있음을 보여줍니다. 대각선(올바른 예측)은 카테고리당 1-4개의 올바른 분류만 보여주고, 오류는 무작위로 분포되어 있습니다.

이는 약 10%의 정확도라는 기준선을 확인합니다—10개 카테고리에서 순전히 우연에 의한 성능입니다. 구조화된 분류 접근 방식은 훨씬 더 강한 대각선 패턴을 보여야 하며, 이는 모델이 추측이 아닌 실제로 카테고리 구분을 학습하고 있음을 나타냅니다.

### 간단한 분류 테스트

이제 Claude를 사용하여 간단한 분류기를 구성해봅시다.

먼저 카테고리를 XML 형식으로 인코딩합니다. 이렇게 하면 Claude가 정보를 더 쉽게 해석할 수 있습니다. 정보를 XML로 인코딩하는 것은 일반적인 프롬프팅 전략입니다. 자세한 내용은 [여기](https://docs.claude.com/en/docs/build-with-claude/prompt-engineering/use-xml-tags)를 참조하세요.

### Simple Classification Test

Now lets construct a simple classifier using Claude.

First we will encode the categories in XML format. This will make it easier for Claude to interpret the information. Encoding information in XML is a general prompting strategy, for more information [see here](https://docs.claude.com/en/docs/build-with-claude/prompt-engineering/use-xml-tags)

## 간단한 분류기 구축하기

이제 Claude를 사용하여 첫 번째 실제 분류기를 만들어봅시다. `simple_classify` 함수는 세 가지 핵심 프롬프트 엔지니어링 기법을 보여줍니다:

1. **구조화된 프롬프트 템플릿**: 카테고리 정의와 지원 티켓을 명확한 XML 형식으로 제공하여 Claude가 정보를 쉽게 파싱할 수 있게 합니다.

2. **프리필링을 통한 출력 제어**: 어시스턴트의 응답을 `<category>`로 시작하고 `stop_sequences=["</category>"]`를 설정함으로써 Claude가 카테고리 레이블만 출력하도록 강제합니다—설명이나 추가 텍스트 없이. 이렇게 하면 응답 파싱이 신뢰할 수 있고 결정론적이 됩니다.

3. **결정론적 분류**: `temperature=0.0`을 설정하면 동일한 입력에 대해 일관된 예측을 보장하며, 이는 분류 작업에 매우 중요합니다.

## Building the Simple Classifier

Now let's build our first real classifier using Claude. The `simple_classify` function demonstrates three key prompt engineering techniques:

1. **Structured prompt template**: We provide the category definitions and the support ticket in a clear XML format, making it easy for Claude to parse the information.

2. **Controlled output with prefilling**: By starting the assistant's response with `<category>` and setting `stop_sequences=["</category>"]`, we force Claude to output just the category label—no explanation or extra text. This makes response parsing reliable and deterministic.

3. **Deterministic classification**: Setting `temperature=0.0` ensures consistent predictions for the same input, which is critical for classification tasks.


In [9]:
def simple_classify(X):
    prompt = (
        textwrap.dedent("""
    You will classify a customer support ticket into one of the following categories:
    <categories>
        {{categories}}
    </categories>

    Here is the customer support ticket:
    <ticket>
        {{ticket}}
    </ticket>

    Respond with just the label of the category between category tags.
    """)
        .replace("{{categories}}", categories)
        .replace("{{ticket}}", X)
    )
    response = client.messages.create(
        messages=[
            {"role": "user", "content": prompt},
            {"role": "assistant", "content": "<category>"},
        ],
        stop_sequences=["</category>"],
        max_tokens=4096,
        temperature=0.0,
        model=MODEL,
    )

    # Extract the result from the response
    result = response.content[0].text  # pyright: ignore[reportAttributeAccessIssue]
    return result.strip()

훨씬 나아졌습니다! 혼동 행렬은 대부분의 카테고리가 높은 정확도를 달성하며 강한 대각선 패턴을 보여줍니다.

이 ~70%의 전체 정확도는 무작위 추측보다 훨씬 뛰어나지만, 유사한 카테고리 간의 혼동은 Claude가 더 세밀한 구분을 하기 위해 더 많은 맥락이 필요함을 시사합니다.

이러한 혼동 패턴을 해결하기 위해, 학습 데이터에서 관련 예시를 제공하는 **검색 증강 생성(RAG)**을 추가합니다. 작동 방식은 다음과 같습니다:

1. **학습 예시 임베딩**: VoyageAI의 임베딩 모델을 사용하여 68개의 모든 학습 티켓을 벡터 임베딩으로 변환
2. **시맨틱 검색**: 각 새 티켓에 대해 코사인 유사도를 기반으로 가장 유사한 5개의 학습 예시 찾기
3. **프롬프트 보강**: 이러한 유사한 예시를 분류 프롬프트에 포함하여 Claude 안내

이 접근 방식은 분류에 특히 효과적입니다:
- 유사한 과거 예시가 Claude가 의미적으로 가까운 카테고리를 구분하는 데 도움
- 파인튜닝 없이 Few-shot 학습으로 정확도 향상
- 검색이 동적임—각 쿼리가 가장 관련 있는 예시를 얻음

[VoyageAI의 임베딩 모델](https://docs.claude.com/en/docs/embeddings)을 사용하여 임베딩 저장 및 유사성 검색을 처리하는 간단한 `VectorDB` 클래스를 구축합니다.

Much better! The confusion matrix shows a strong diagonal pattern with most categories achieving high accuracy. 

This ~70% overall accuracy significantly outperforms random guessing, but the confusion between similar categories suggests Claude needs more context to make finer distinctions.

To address these confusion patterns, we'll add **retrieval-augmented generation (RAG)** by providing Claude with relevant examples from our training data. Here's how it works:

1. **Embed training examples**: Convert all 68 training tickets into vector embeddings using VoyageAI's embedding model
2. **Semantic search**: For each new ticket, find the 5 most similar training examples based on cosine similarity
3. **Augment the prompt**: Include these similar examples in the classification prompt to guide Claude

This approach is particularly effective for classification because:
- Similar past examples help Claude distinguish between semantically close categories
- Few-shot learning improves accuracy without fine-tuning
- The retrieval is dynamic—each query gets the most relevant examples

We'll build a simple `VectorDB` class to handle embedding storage and similarity search using [VoyageAI's embedding models](https://docs.claude.com/en/docs/embeddings).

벡터 DB를 정의하고 학습 데이터를 로드할 수 있습니다.

VoyageAI는 신용카드가 연결되지 않은 계정에 대해 3RPM의 rate limit이 있습니다. 시연의 편의를 위해 캐시를 활용합니다.

We can define the vector db and load our training data.

VoyageAI has a rate limit of 3RPM for accounts without an associated credit card. For ease of demonstration we will leverage a cache.

## RAG 강화 분류기

이제 검색 증강 생성을 적용한 분류기를 다시 구축합니다. `rag_classify` 함수는 다음과 같이 간단한 분류기를 향상시킵니다:

1. **유사한 예시 검색**: 각 티켓에 대해 벡터 데이터베이스에서 의미적으로 가장 유사한 5개의 학습 예시를 검색
2. **Few-shot 예시 형식화**: `<query>`와 `<label>` 태그가 있는 XML 형식으로 이러한 예시를 구조화하여 유사한 티켓이 어떻게 분류되었는지 Claude에게 보여줌
3. **프롬프트 보강**: 새 티켓을 분류하기 전에 프롬프트에 이러한 예시를 주입

이 Few-shot 학습 접근 방식은 유사한 언어를 가진 티켓이 어떻게 분류되어야 하는지에 대한 구체적인 예시를 보여줌으로써 Claude가 유사한 카테고리 간에 더 나은 구분을 하도록 돕습니다. 예를 들어, 테스트 티켓이 "예상치 못한 청구"를 언급하면, 과거의 "Billing Disputes" vs "Billing Inquiries" 예시를 검색하여 Claude가 미묘한 차이를 이해하도록 돕습니다.

이전 실행에서 혼동된 카테고리에 대해 정확도가 얼마나 향상되는지 살펴봅시다.

## RAG-Enhanced Classifier

Now let's rebuild our classifier with retrieval-augmented generation. The `rag_classify` function enhances the simple classifier by:

1. **Retrieving similar examples**: For each ticket, we search the vector database for the 5 most semantically similar training examples
2. **Formatting as few-shot examples**: We structure these examples in XML format with `<query>` and `<label>` tags, showing Claude how similar tickets were classified
3. **Augmenting the prompt**: We inject these examples into the prompt before asking Claude to classify the new ticket

This few-shot learning approach helps Claude make better distinctions between similar categories by showing concrete examples of how tickets with similar language should be categorized. For instance, if the test ticket mentions "unexpected charges," retrieving examples of past "Billing Disputes" vs "Billing Inquiries" helps Claude understand the subtle difference.

Let's see how much this improves our accuracy on the confused categories from the previous run.

In [13]:
def rag_classify(X):
    rag = vectordb.search(X, 5)
    rag_string = ""
    for example in rag:
        rag_string += textwrap.dedent(f"""
        <example>
            <query>
                "{example["metadata"]["text"]}"
            </query>
            <label>
                {example["metadata"]["label"]}
            </label>
        </example>
        """)
    prompt = (
        textwrap.dedent("""
    You will classify a customer support ticket into one of the following categories:
    <categories>
        {{categories}}
    </categories>

    Here is the customer support ticket:
    <ticket>
        {{ticket}}
    </ticket>

    Use the following examples to help you classify the query:
    <examples>
        {{examples}}
    </examples>

    Respond with just the label of the category between category tags.
    """)
        .replace("{{categories}}", categories)
        .replace("{{ticket}}", X)
        .replace("{{examples}}", rag_string)
    )
    response = client.messages.create(
        messages=[
            {"role": "user", "content": prompt},
            {"role": "assistant", "content": "<category>"},
        ],
        stop_sequences=["</category>"],
        max_tokens=4096,
        temperature=0.0,
        model="claude-haiku-4-5",
    )

    # Extract the result from the response
    result = response.content[0].text.strip()
    return result

## RAG 결과 분석

RAG가 정확도를 ~70%에서 **94%**로 향상시켰습니다. 혼동 행렬은 훨씬 더 강한 대각선을 보여주며, 대부분의 카테고리가 이제 완벽하거나 거의 완벽한 정확도를 달성했습니다.

남은 몇 가지 오류는 관련 예시가 있어도 일부 엣지 케이스가 여전히 모호하다는 것을 시사합니다. 여기서 Chain-of-Thought 추론이 Claude가 미묘한 구분을 더 신중하게 생각하도록 도울 수 있습니다.

## RAG와 Chain-of-Thought 추론

Chain-of-Thought (CoT) 프롬프팅은 Claude에게 분류 결정을 내리기 전에 "소리 내어 생각"하도록 요청합니다.

이 명시적인 추론 과정은 직접 분류에서 놓칠 수 있는 미묘한 구분을 Claude가 포착하도록 돕습니다. 예를 들어, "청구서에 대해 질문이 있습니다"(Billing Inquiry)와 "이 청구가 잘못된 것 같습니다"(Billing Dispute)를 구분하려면 의도와 어조를 이해해야 하며—이는 단계별 분석의 이점을 얻습니다.

`rag_chain_of_thought_classify` 함수는 Claude가 최종 `<category>`를 출력하기 전에 이 추론을 수행하는 `<scratchpad>` 섹션을 추가합니다.

## RAG with Chain-of-Thought Reasoning

Chain-of-thought (CoT) prompting asks Claude to "think out loud" before making a classification decision. 

This explicit reasoning process helps Claude catch subtle distinctions that might be missed with direct classification. For example, distinguishing "I have a question about my bill" (Billing Inquiry) from "This charge seems wrong" (Billing Dispute) requires understanding intent and tone—something that benefits from step-by-step analysis.

The `rag_chain_of_thought_classify` function adds a `<scratchpad>` section where Claude works through this reasoning before outputting the final `<category>`.

In [ ]:
def rag_chain_of_thought_classify(X):
    rag = vectordb.search(X, 5)
    rag_string = ""
    for example in rag:
        rag_string += textwrap.dedent(f"""
        <example>
            <query>
                "{example["metadata"]["text"]}"
            </query>
            <label>
                {example["metadata"]["label"]}
            </label>
        </example>
        """)
    prompt = (
        textwrap.dedent("""
    You will classify a customer support ticket into one of the following categories:
    <categories>
        {{categories}}
    </categories>

    Here is the customer support ticket:
    <ticket>
        {{ticket}}
    </ticket>

    Use the following examples to help you classify the query:
    <examples>
        {{examples}}
    </examples>

    First you will think step-by-step about the problem in scratchpad tags.
    You should consider all the information provided and create a concrete argument for your classification.

    Respond using this format:
    <response>
        <scratchpad>Your thoughts and analysis go here</scratchpad>
        <category>The category label you chose goes here</category>
    </response>
    """)
        .replace("{{categories}}", categories)
        .replace("{{ticket}}", X)
        .replace("{{examples}}", rag_string)
    )
    response = client.messages.create(
        messages=[
            {"role": "user", "content": prompt},
            {"role": "assistant", "content": "<response><scratchpad>"},
        ],
        stop_sequences=["</category>"],
        max_tokens=4096,
        temperature=0.0,
        model=MODEL,
    )

    # Extract the result from the response
    result = response.content[0].text.split("<category>")[1].strip()
    return result

## Chain-of-Thought 결과 분석

Chain-of-Thought 추론이 정확도를 **97%**로 끌어올려, 이전 접근 방식들이 어려워했던 대부분의 엣지 케이스를 해결했습니다.

**점진적 개선 요약:**
- 랜덤 기준선: ~10% 정확도
- 간단한 분류기: ~70% 정확도
- RAG 분류기: 94% 정확도  
- **RAG + Chain-of-Thought: 97% 정확도**

각 분류 결정을 명시적으로 추론함으로써 Claude는 모호한 케이스 간에 더 나은 구분을 합니다.

# Promptfoo를 사용한 대규모 평가

이 가이드 전체에서 프롬프트를 엔지니어링할 때 **경험적 평가**의 중요성을 보여주었습니다. 직관에 의존하는 대신 각 단계에서 성능을 측정했습니다:

- 랜덤 기준선: ~10% → Simple: ~70% → RAG: 94% → RAG + CoT: 97%

이 데이터 기반 접근 방식은 어떤 기법(RAG, Chain-of-Thought)이 가치를 추가하는지와 얼마나 많이 추가하는지를 정확히 보여주었습니다. 이 방법론에 대한 자세한 내용은 [프롬프트 엔지니어링 가이드](https://docs.claude.com/en/docs/prompt-engineering)를 참조하세요.

## 노트북을 넘어서

Jupyter 노트북은 빠른 반복과 탐색에 탁월하지만, 프로덕션 분류 시스템에는 더 견고한 평가 인프라가 필요합니다:

- **더 큰 테스트 세트**: 68개가 아닌 수백 또는 수천 개의 예시로 평가하고 싶을 것입니다
- **여러 프롬프트 변형**: 다양한 표현, 구조, 접근 방식의 A/B 테스트
- **모델 비교**: 다양한 Claude 모델(Haiku vs Sonnet)이나 temperature 설정에 대한 테스트
- **회귀 감지**: 프롬프트 변경이 실수로 정확도를 손상시키지 않는지 확인
- **버전 제어**: 시스템이 발전함에 따라 시간 경과에 따른 프롬프트 성능 추적

이것이 [Promptfoo](https://www.promptfoo.dev/)와 같은 전용 평가 도구가 필수적인 이유입니다.

## Promptfoo 평가 실행

Promptfoo는 여러 구성에 걸쳐 프롬프트 테스트를 자동화하는 오픈소스 LLM 평가 툴킷입니다. 이 가이드에서는 세 가지 접근 방식(Simple, RAG, RAG w/ CoT)을 다양한 temperature 설정에서 테스트하는 Promptfoo 평가를 설정했습니다.

**평가 실행 방법:**

1. 평가 디렉토리로 이동: `cd ./evaluation`
2. `./evaluation/README.md`의 설정 지침을 따르기
3. 평가를 실행하고 아래에서 결과를 분석하기 위해 여기로 돌아오기

결과는 각 프롬프트 변형이 전체 테스트 세트에서 어떻게 수행되는지 보여주며, 노트북 테스트에서는 명확하지 않을 수 있는 패턴을 드러냅니다.

# Scaling Evaluation with Promptfoo

Throughout this guide, we've demonstrated the importance of **empirical evaluation** when engineering prompts. Rather than relying on intuition, we measured performance at each step:

- Random baseline: ~10% → Simple: ~70% → RAG: 94% → RAG + CoT: 97%

This data-driven approach revealed exactly which techniques (RAG, chain-of-thought) added value and by how much. For more on this methodology, see our [prompt engineering guide](https://docs.claude.com/en/docs/prompt-engineering).

## Moving Beyond Notebooks

While Jupyter notebooks are excellent for rapid iteration and exploration, production classification systems require more robust evaluation infrastructure:

- **Larger test sets**: You'll want to evaluate on hundreds or thousands of examples, not just 68
- **Multiple prompt variants**: A/B testing different phrasings, structures, and approaches
- **Model comparisons**: Testing across different Claude models (Haiku vs Sonnet) or temperature settings
- **Regression detection**: Ensuring prompt changes don't accidentally hurt accuracy
- **Version control**: Tracking prompt performance over time as your system evolves

This is where dedicated evaluation tools like [Promptfoo](https://www.promptfoo.dev/) become essential.

## Running Promptfoo Evaluation

Promptfoo is an open-source LLM evaluation toolkit that automates prompt testing across multiple configurations. For this guide, we've set up a Promptfoo evaluation that tests all three approaches (Simple, RAG, RAG w/ CoT) across different temperature settings.

**To run the evaluation:**

1. Navigate to the evaluation directory: `cd ./evaluation`
2. Follow the setup instructions in `./evaluation/README.md`
3. Run the evaluation and return here to analyze the results below

The results will show how each prompt variant performs across the full test set, revealing patterns that might not be obvious from notebook testing alone.

## Promptfoo를 사용한 체계적인 평가

위의 결과는 대규모 체계적인 프롬프트 평가의 힘을 보여줍니다. 노트북이 개발 중 빠른 반복을 제공했지만, [Promptfoo](https://www.promptfoo.dev/)를 통해 여러 차원에 걸쳐 프롬프트를 엄격하게 테스트할 수 있었습니다:

### 테스트 내용

Promptfoo를 사용하여 세 가지 프롬프트 접근 방식(Simple, RAG, RAG w/ CoT)을 다음에 걸쳐 평가했습니다:
- **여러 temperature 설정** (0.0, 0.2, 0.4, 0.6, 0.8): 결정론적 분류(T=0.0)가 약간 무작위화된 출력보다 나은 성능을 보이는지 테스트
- **동일한 테스트 세트** (68개 예시): 모든 구성에서 공정한 비교 보장
- **자동화된 합격/불합격 검사**: 각 예측이 자동으로 정답 레이블과 비교됨

### 주요 발견

Promptfoo 결과는 노트북 결과를 확인하면서 추가적인 인사이트를 드러냅니다:

1. **Temperature가 CoT에 미치는 영향 최소화**: RAG w/ CoT는 T=0.0, 0.2, 0.8에서 일관되게 95.59%의 정확도를 달성하여, Chain-of-Thought 추론이 샘플링 무작위성에 관계없이 출력을 안정화함을 시사

2. **RAG는 temperature 변화에 견고함**: RAG 성능은 대부분의 temperature 설정에서 강력하게 유지(89-94%)되지만 T=0.0이 가장 좋은 성능(94.12%)

3. **Simple 프롬프트는 temperature에 무관**: 간단한 분류기는 temperature에 관계없이 약 70%의 정확도를 유지하며, RAG나 CoT 없이는 모델이 주로 카테고리 정의에 의존함을 확인

4. **프로덕션 권장사항**: 최대 일관성과 정확도(95.59%)를 위해 RAG w/ CoT와 함께 `temperature=0.0` 사용

이 평가를 직접 실행하려면 `./evaluation/README.md`의 설정 지침을 참조하세요.

## Systematic Evaluation with Promptfoo

The results above demonstrate the power of systematic prompt evaluation at scale. While our notebook provided quick iteration during development, [Promptfoo](https://www.promptfoo.dev/) enabled us to rigorously test our prompts across multiple dimensions:

### What We Tested

Using Promptfoo, we evaluated all three prompt approaches (Simple, RAG, RAG w/ CoT) across:
- **Multiple temperature settings** (0.0, 0.2, 0.4, 0.6, 0.8): Testing whether deterministic classification (T=0.0) outperforms slightly randomized outputs
- **The same test set** (68 examples): Ensuring fair comparison across all configurations
- **Automated pass/fail checking**: Each prediction is automatically compared against ground truth labels

### Key Findings

The Promptfoo results confirm our notebook findings while revealing additional insights:

1. **Temperature has minimal impact on CoT**: RAG w/ CoT achieves 95.59% accuracy consistently across T=0.0, 0.2, and 0.8, suggesting the chain-of-thought reasoning stabilizes outputs regardless of sampling randomness

2. **RAG is robust to temperature variation**: RAG performance stays strong (89-94%) across most temperature settings, though T=0.0 performs best (94.12%)

3. **Simple prompts are temperature-agnostic**: The simple classifier hovers around 70% accuracy regardless of temperature, confirming that without RAG or CoT, the model relies primarily on category definitions

4. **Production recommendation**: Use `temperature=0.0` with RAG w/ CoT for maximum consistency and accuracy (95.59%)

To run these evaluations yourself, see `./evaluation/README.md` for setup instructions.